In [0]:
#!/usr/bin/env python

In [0]:
#Import Liabraries
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns;
import pandas.util.testing as tm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import plot_model
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.utils import np_utils

In [0]:
# #Cleaning Dataset and Exporting as cvs
# data_attack = pd.read_csv('dataset_attack.csv')
# data_normal = pd.read_csv('dataset_normal.csv')
# data_normal.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
#        'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
#        'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
#        'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
#        'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
#        'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
#        'tcp.time_delta','class']
# data_attack.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
#        'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
#        'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
#        'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
#        'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
#        'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
#        'tcp.time_delta','class']
# data_normal.to_csv (r'dataset_normal_cleaned.csv', index=False, header=True)
# data_attack.to_csv (r'dataset_attack_cleaned.csv', index=False, header=True)

## Import Dataset from Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = "/content/drive/My Drive/Datasets/dataset_attack_cleaned.csv"
data_attack = pd.read_csv(path)
data_attack.head()

,frame.len,frame.protocols,ip.hdr_len,ip.len,ip.flags.rb,ip.flags.df,p.flags.mf,ip.frag_offset,ip.ttl,ip.proto,ip.src,ip.dst,tcp.srcport,tcp.dstport,tcp.len,tcp.ack,tcp.flags.res,tcp.flags.ns,tcp.flags.cwr,tcp.flags.ecn,tcp.flags.urg,tcp.flags.ack,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.fin,tcp.window_size,tcp.time_delta,class
0,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,178.95.241.121,192.168.2.107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,attack
1,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,178.95.241.121,192.168.2.107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,attack
2,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,178.95.241.121,192.168.2.107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,attack
3,212,eth:ethertype:ip:udp:data,20,198,0,0,0,0,112,17,178.95.241.121,192.168.2.107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,attack
4,62,eth:ethertype:ip:udp:data,20,48,0,0,0,0,128,17,192.168.2.107,178.95.241.121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,attack


In [0]:
path = "/content/drive/My Drive/Datasets/dataset_normal_cleaned.csv"
data_normal = pd.read_csv(path)
data_normal.head()

,frame.len,frame.protocols,ip.hdr_len,ip.len,ip.flags.rb,ip.flags.df,p.flags.mf,ip.frag_offset,ip.ttl,ip.proto,ip.src,ip.dst,tcp.srcport,tcp.dstport,tcp.len,tcp.ack,tcp.flags.res,tcp.flags.ns,tcp.flags.cwr,tcp.flags.ecn,tcp.flags.urg,tcp.flags.ack,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.fin,tcp.window_size,tcp.time_delta,class
0,206,eth:ethertype:ip:tcp:ssh,20,192,0,1,0,0,128,6,192.168.1.101,192.168.5.122,4175,22,152,1,0,0,0,0,0,1,1,0,0,0,16697,0.000000,normal
1,60,eth:ethertype:ip:tcp,20,40,0,1,0,0,64,6,192.168.5.122,192.168.1.101,22,4175,0,153,0,0,0,0,0,1,0,0,0,0,7504,0.000537,normal
2,60,eth:ethertype:ip:tcp,20,40,0,1,0,0,63,6,192.168.5.122,192.168.1.101,22,4175,0,153,0,0,0,0,0,1,0,0,0,0,7504,0.000155,normal
3,774,eth:ethertype:ip:tcp:ssh,20,760,0,1,0,0,64,6,192.168.5.122,192.168.1.101,22,4175,720,153,0,0,0,0,0,1,1,0,0,0,7504,0.004483,normal
4,774,eth:ethertype:ip:tcp,20,760,0,1,0,0,63,6,192.168.5.122,192.168.1.101,22,4175,720,153,0,0,0,0,0,1,1,0,0,0,7504,0.001321,normal


## Data Preprocessing

In [0]:
#Drop Unwanted Columns
data_normal=data_normal.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)
data_attack=data_attack.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)


In [0]:
# Feature Selection for training
features = data_attack.head(0)
features=data_attack.columns.tolist()
features.remove("class")
print(features)

['frame.len', 'ip.hdr_len', 'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset', 'ip.ttl', 'ip.proto', 'tcp.srcport', 'tcp.dstport', 'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr', 'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size', 'tcp.time_delta']


In [0]:
#Combinig dataset for training and testing
X_normal= data_normal[features].values
X_attack= data_attack[features].values
Y_normal= data_normal['class']
Y_attack= data_attack['class']
X=np.concatenate((X_normal,X_attack))
Y=np.concatenate((Y_normal,Y_attack))

In [0]:
#Standardise Data to scale Numerical features
scalar = StandardScaler(copy=True, with_mean=True, with_std=True)
scalar.fit(X)
X = scalar.transform(X)


In [0]:
#Replace attack and normal with 0 and 1 respectively
for i in range(0,len(Y)):
  if Y[i] =="attack":
    Y[i]=0
  else:
    Y[i]=1

In [0]:
#Split data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [0]:
# create the model
def create_baseline():
    model = Sequential()
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

